# Exploring sales advertising in the housing market in Saint Pietersburg

The data set for this research is the Yandex.Realty archive of ads for the apartments sal in St. Petersburg and neighboring settlements. 

The main goal of this research is to understand what affects the realty prices. The outcome of the research will be used to build an automated system that will track anomalies and fraudulent activity.

There are two types of data available for each apartment for sale. The first type is the data that was entered by the user, the second type is the data that obtained automatically based on cartography. For example, the distance to the center, airport, nearest park and so on.

The table contains the following data:

* 'airports_nearest` — distance to the nearest airport in meters (m)
* 'balcony` — number of balconies
* 'ceiling_height` — ceiling height (m)
* 'cityCenters_nearest` - distance to the city center (m)
* 'days_exposition` — how many days the ad was placed (from publication to removal)
* 'first_day_exposition` — date of publication
* 'floor` - floor
* 'floors_total` — total floors in the house
* 'is_apartment` - apartments (boolean type)
* 'kitchen_area` — kitchen area in square meters (m2)
* 'last_price` - price at the time of withdrawal from publication
* 'living_area` — living area in square meters(m2)
* 'locality_name` — name of the locality
* 'open_plan` — free layout (boolean type)
* 'parks_around3000` — the number of parks within a 3 km radius
* 'parks_nearest` — distance to the nearest park (m)
* 'ponds_around3000` — number of reservoirs within a radius of 3 km
* 'ponds_nearest` — distance to the nearest reservoir (m)
* 'rooms` — number of rooms
* 'studio` — studio apartment (boolean type)
* 'total_area` — the area of the apartment in square meters (m2)
* 'total_images` — the number of photos of the apartment in the ad

The research includes the following stages:

1. Data investigation.
2. Data cleanup.
3. Data analysis.
4. Follow up.

# Data investigation

First thing to do is to load tha data and check it's shape.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# libraries import
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
!pip install pymystem3==0.1.10
from pymystem3 import Mystem
m = Mystem()
from matplotlib import pyplot as plt


  Attempting uninstall: pymystem3
    Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [3]:
# data import
df = pd.read_csv("/content/drive/MyDrive/da_portfolio/real_estate_data.csv", sep="\t") 
display(df.head(10))
display(df.info())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,...,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,...,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,...,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,...,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,...,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,...,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,...,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,...,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,...,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

None

Now the data is loaded and I can see that not all data can be used for analysis the way it is now: the number of rows in columns is not equal, some cells have `NaN` as a value, also there is a need to check the data for duplicates. 





# Data cleanup

I will start with checking for duplicates.

In [4]:
# converting ;ocality to the lowercase
df["locality_name"] = df["locality_name"].str.lower()

# checking for duplicates in the wholde dataframe
print("The number of duplicates is:", df.duplicated().sum())

The number of duplicates is: 0


Well, now I know that there are no duplicates, so I can move on with my data cleanup. The next step is to decide what to do with unequal number of rows in columns.  

In general unequal number of rows means, that some cells are not filled with the porper data. I will check the amount of such cells per row and will also check if everything is fine with the columns' names.

In [5]:
# checking the columns' names
df.columns

Index(['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'cityCenters_nearest',
       'parks_around3000', 'parks_nearest', 'ponds_around3000',
       'ponds_nearest', 'days_exposition'],
      dtype='object')

No spaces, no camelcase. All fine.

In [6]:
# checking the amount of  null in the column
df.isnull().sum()

total_images                0
last_price                  0
total_area                  0
first_day_exposition        0
rooms                       0
ceiling_height           9195
floors_total               86
living_area              1903
floor                       0
is_apartment            20924
studio                      0
open_plan                   0
kitchen_area             2278
balcony                 11519
locality_name              49
airports_nearest         5542
cityCenters_nearest      5519
parks_around3000         5518
parks_nearest           15620
ponds_around3000         5518
ponds_nearest           14589
days_exposition          3181
dtype: int64

So now I can see that the amount of null values in the dataset is high. Nearly every column has some. Here I decide what to do with such values, and I will start with the understandig what columns I will use for the further analysis.     

It looks like I will need the following columns:

* `ceiling_height`
* `floors_total`
* `locality_name`
* `cityCenters_nearest`
* `days_exposition`
* `living_area`
* `kitchen_area`

So I will run the data cleanup only on them, avoiding doing anything to the other columns.

**Ceiling height** (`ceiling_height`) - currently, the ceiling height in new and old buildings is approximately the same 2.5 - 2.8 m. The exception is "stalinka", where the ceiling height can reach 3+ meters. I do not have the type/year of construction in the table, so there is no category to rely on when filling in the gaps. Therefore, the median of the `ceiling_height` category will be used to fill in the gaps.

In [7]:
# checking the median value
print("The median is:", df["ceiling_height"].median(), "meters")

The median is: 2.65 meters


In [8]:
# filling the gaps with median value
df["ceiling_height"].fillna((df["ceiling_height"].median()), inplace=True)

**Total floors in the house** (`floors_total`) - as well as with the height of the ceilings, data on the year of construction or type of construction is required. In addition, we have the `floor` parameter, which complicates filling in the gaps in this column: I can fill in the gap in `floors_total` with a median equal to 9 while in the `floor` column it will be indicated that the client lives on the 13th floor.

In order not to complicate the task, I'm looking at the percentage of omissions in `floors_total`, which is 0.36% of the total number of rows. This is a small percentage, so lines with omissions can be deleted.

In order to be able to apply some functions to the column values, I change the data type to numeric.

In [9]:
# deleting NaN
df = df.dropna(subset=["floors_total"])

# changing the cityCenters_nearest datatype to 'int'
df["floors_total"] = df["floors_total"].astype("int")


**The name of the locality** (`locality_name`) - just like with the number of floors of houses, the number of lines with gaps is small, there are no parameters to rely on to fill them in, so I delete the lines with gaps.

In [10]:
# deleting NaN
df = df.dropna(subset=["locality_name"])

**Proximity to the center** (`city Center s_nearest`) - gaps in the data here ara due to the fact that `citycenter_nearest` cannot be applied to some of the settlments. For example, some villages and towns just do not have the "city center" since they are too small, so there is little difference between the center and the outskirts and remoteness does not affect the price.


For towns and villages that are sattelites of the bigger city, it is possible to indicate the remotness from the main city center, and not to the center of the settlement itself.

In [11]:
# I start with the list of the settlments
df["locality_name"].unique()

array(['санкт-петербург', 'посёлок шушары', 'городской посёлок янино-1',
       'посёлок парголово', 'посёлок мурино', 'ломоносов', 'сертолово',
       'петергоф', 'пушкин', 'деревня кудрово', 'коммунар', 'колпино',
       'поселок городского типа красный бор', 'гатчина', 'поселок мурино',
       'деревня фёдоровское', 'выборг', 'кронштадт', 'кировск',
       'деревня новое девяткино', 'посёлок металлострой',
       'посёлок городского типа лебяжье',
       'посёлок городского типа сиверский', 'поселок молодцово',
       'поселок городского типа кузьмоловский',
       'садовое товарищество новая ропша', 'павловск',
       'деревня пикколово', 'всеволожск', 'волхов', 'кингисепп',
       'приозерск', 'сестрорецк', 'деревня куттузи', 'посёлок аннино',
       'поселок городского типа ефимовский', 'посёлок плодовое',
       'деревня заклинье', 'поселок торковичи', 'поселок первомайское',
       'красное село', 'посёлок понтонный', 'сясьстрой', 'деревня старая',
       'деревня лесколово', '

In [12]:
#  and checking the amount of items in each category
df["locality_name"].value_counts()

санкт-петербург                    15651
посёлок мурино                       520
посёлок шушары                       439
всеволожск                           398
пушкин                               369
                                   ...  
поселок каложицы                       1
посёлок платформа 69-й километр        1
поселок почап                          1
поселок гончарово                      1
посёлок дзержинского                   1
Name: locality_name, Length: 364, dtype: int64

There are many settlements, but some of them have only 1 value. 

Also, from the names of the cities and towns I can see that mostly all of them are placed around Saint Petersburg. I can understand the size of the settlment from it's name: "village", "settlement", "urban-type settlement" and so on. Cities are recorded simply by names, without additional clarifications.

In order to make it easier to analyze the data, I will leave only the names of localities and lemmatize them to divide into categories.

In [13]:
# lemmatization with categorizing function
def lemm(liveplace):
    lemma = ' '.join(m.lemmatize(liveplace))
    if "поселок город тип" in lemma:
        return lemma.replace("посёлок город тип", '')
    elif "деревня" in lemma:
        return lemma.replace("деревня", '')
    elif "поселок" in lemma:
        return lemma.replace("поселок", '')
    elif "село" in lemma:
        return lemma.replace("село", '')
    elif "товарищество" in lemma:
         return lemma.replace("товарищество", '')
    else:
        pass
    return lemma

# applying to the df and add the result to locality_name_lem
df["locality_name_lem"] = df["locality_name"].apply(lemm)


In [14]:
# and checking the results
df["locality_name_lem"].value_counts()

санкт-петербург \n                     15651
   муриный \n                            552
   шушары \n                             439
всеволожск \n                            398
пушкин \n                                369
                                       ...  
   пельгора \n                             1
   каложица \n                             1
   платформа   69 - й   километр \n        1
   почап \n                                1
   дзержинский \n                          1
Name: locality_name_lem, Length: 325, dtype: int64

Now I check data by category.

In [15]:
# data check by category
df.groupby("locality_name_lem")["cityCenters_nearest"].agg([min, max, sum])

,min,max,sum
locality_name_lem,,,
агалатово \n,NaN,NaN,0.0
александровский \n,27352.0,27584.0,54936.0
алексеевка \n,NaN,NaN,0.0
аннино \n,NaN,NaN,0.0
аро \n,NaN,NaN,0.0
...,...,...,...
сосновый бор \n,NaN,NaN,0.0
сясьстрой \n,NaN,NaN,0.0
тихвин \n,NaN,NaN,0.0


From the table above I can see, that the gaps are related with the settlments that are not big enough to have such a parameter as `distance` from the center. In large cities, such as St. Petersburg, Pavlovsk, etc - the `distance` from the center is indicated.

The `sum` column shows that in some categories there is no data for the entire category, which means I have no values to calculate the median.

Filling in the median across the entire column does not seem to be the right solution because in some cases cases the median may be bigger than the settlement diameter.
makes 1.3% of the total number of rows. Since the percentage is small, I delete the lines with gaps so that it will be more convenient to analyze the data in the future.

In [16]:
# deleting the null values
df.dropna(subset=["cityCenters_nearest"], inplace=True)

#checking the duplicates
print("The number of duplicates is:", df["cityCenters_nearest"].isnull().sum())

The number of duplicates is: 0


**Ad placement period** (`days_exposition`) - the reason for gaps in this column are still active ads or ads that have been published for less than a day. 

I can't fill such gaps with zeros or another value, but their presence may affect the analysis, so I delete them from the table.

In [17]:
# deleting the null values
df['days_exposition'] = df['days_exposition'].dropna()

****Living area** (`living_area`) is the total area of rooms intended for living. 

To the gaps in this column, I will ckeck how the living area of other apartments (in the columns without gaps) correlates with the number of rooms, and fill in the gaps with the median.

In [18]:
# checking the number o f rooms
df['rooms'].value_counts()

2     5888
1     5852
3     4667
4     1051
5      300
0      117
6      101
7       57
8       12
9        8
10       3
11       2
14       2
15       1
19       1
16       1
12       1
Name: rooms, dtype: int64

So, 0.2% of rooms are 0, and 2% (in total) are rooms with 5+ rooms. 

It looks like some kind of mistake occured when data was collected. For example, 0 might be an idicator for studio apratmens, where the  where the living room is combined with the kitchen and, on the other hand, someone counted toilet + kitchen + etc as "rooms" therefore we have a lot of 5+ rooms apts. Or, maybe, someone is selling a whole multi-storey house.

The amount of such data is not statisticaly significant and can be treated as outliers. I am removing this data not to make the further analysis difficult. 


In [20]:
# removing all lines with rooms more than 5
df = df.drop(df[df.rooms >= 5].index)

# removing all lines with rooms  0
df = df.drop(df[df.rooms == 0].index)

# replacing with median by 'rooms' category
df['living_area']=df.groupby('rooms')['living_area'].transform(lambda x: x.fillna(x.median()))

# updating data type to ('int')
df['living_area'] = df['living_area'].astype('int')